In [7]:
def adj_to_coladj(adj_mat, dynamics):
    col_adj = np.zeros((adj_mat.shape[0], adj_mat.shape[1]))
    for i in range(adj_mat.shape[0]):
        for j in range(i, adj_mat.shape[1]):
            if i == j or adj_mat[i][j] == 0:
                col_adj[i][j] = 0
                col_adj[j][i] = 0
            if adj_mat[i][j] == 1:
                a = abs(dynamics[j] - dynamics[i])
                b = abs(2*np.pi + (dynamics[j] - dynamics[i]))
                col_adj[i][j] = min(a, b)
                col_adj[j][i] = min(a, b)
                
    return col_adj

In [8]:
# model = Kuramoto(
#     coupling=10,
#     dt=0.05,
#     T=10000,
#     n_nodes=50,
#     base_iter=126,
#     half_sync=False
# )

In [9]:
# G = nx.newman_watts_strogatz_graph(50, 10, 0.35)
# adj = nx.to_numpy_array(G)
# sim = model.run(adj)

In [10]:
# model.concentrated

In [11]:
# sns.heatmap(adj);

In [12]:
# dynamics = sim.T[0]
# col_adj = adj_to_coladj(adj, dynamics)
# sns.heatmap(col_adj);

In [13]:
# dynamics = sim.T[125]
# col_adj = adj_to_coladj(adj, dynamics)
# sns.heatmap(col_adj);

In [14]:
# dynamics = sim.T[-1]
# col_adj = adj_to_coladj(adj, dynamics)
# sns.heatmap(col_adj);

In [15]:
# model = Kuramoto(
#     coupling=10,
#     dt=0.01,
#     T=18,
#     n_nodes=50,
#     base_iter=126,
#     half_sync=False
# )

In [16]:
# G = nx.grid_2d_graph(8, 8)
# adj = nx.to_numpy_array(G)

In [17]:
# model = Kuramoto(
#     coupling=10,
#     dt=0.01,
#     T=18,
#     n_nodes=64,
#     base_iter=126,
#     half_sync=False
# )

# sim = model.run(adj)

In [18]:
# sns.heatmap(sim.T[0].reshape(8, 8), cbar=False, xticklabels=False, yticklabels=False);

In [19]:
# sns.heatmap(sim.T[65].reshape(8, 8), cbar=False, xticklabels=False, yticklabels=False);

In [20]:
# sns.heatmap(sim.T[130].reshape(8, 8), cbar=False, xticklabels=False, yticklabels=False);

In [21]:
# sns.heatmap(sim.T[300].reshape(8, 8), cbar=False, xticklabels=False, yticklabels=False);

In [22]:
# model.baseline

In [23]:
# sim.T[-1].reshape(10, 8)

In [24]:
# sim.T[-1]

In [25]:
NWS = nx.newman_watts_strogatz_graph(20000, 1000, 0.67)
new_nodes = {e: n for n, e in enumerate(NWS.nodes, start=1)}
new_edges = [(new_nodes[e1], new_nodes[e2]) for e1, e2 in NWS.edges]
edgelist = []
for i in range(len(new_edges)):
    temp = [str(new_edges[i][0]), str(new_edges[i][1])]
    edgelist.append(temp)
G = nn.NNetwork()
G.add_edges(edgelist)

In [27]:
NWS.number_of_edges()

16704084

In [28]:
NWS.number_of_nodes()

20000

In [29]:
nx.density(NWS)

0.0835245962298115

In [30]:
nx.average_clustering(NWS)

0.3091395308962197